In [1]:
import keras
import numpy as np
import nltk
from nltk.corpus import cmudict
from nltk.corpus import wordnet as wn
import re

Using TensorFlow backend.


## Reading data

In [2]:
def get_data():
    words = []
    with open("words.txt") as fi:
        line = fi.readline()
        while(len(line)!=0):
            datapoint = line.split(' ')
            phon = datapoint[1].split('.')
            phon[-1] = phon[-1][:-1]
            words.append([datapoint[0], phon])
            line = fi.readline()
            
    return(words)

In [3]:
data = get_data()

In [19]:
data[:20]

[['aback', ['AH', 'B', 'AE', 'K']],
 ['abandon', ['AH', 'B', 'AE', 'N', 'D', 'AH', 'N']],
 ['abandoned', ['AH', 'B', 'AE', 'N', 'D', 'AH', 'N', 'D']],
 ['abandoning', ['AH', 'B', 'AE', 'N', 'D', 'AH', 'N', 'IH', 'NG']],
 ['abandons', ['AH', 'B', 'AE', 'N', 'D', 'AH', 'N', 'Z']],
 ['abash', ['AH', 'B', 'AE', 'SH']],
 ['abashed', ['AH', 'B', 'AE', 'SH', 'T']],
 ['abate', ['AH', 'B', 'EY', 'T']],
 ['abated', ['AH', 'B', 'EY', 'T', 'IH', 'D']],
 ['abates', ['AH', 'B', 'EY', 'T', 'S']],
 ['abating', ['AH', 'B', 'EY', 'T', 'IH', 'NG']],
 ['abbreviate', ['AH', 'B', 'R', 'IY', 'V', 'IY', 'EY', 'T']],
 ['abbreviates', ['AH', 'B', 'R', 'IY', 'V', 'IY', 'EY', 'T', 'S']],
 ['abbreviating', ['AH', 'B', 'R', 'IY', 'V', 'IY', 'EY', 'T', 'IH', 'NG']],
 ['abdicate', ['AE', 'B', 'D', 'AH', 'K', 'EY', 'T']],
 ['abdicated', ['AE', 'B', 'D', 'AH', 'K', 'EY', 'T', 'AH', 'D']],
 ['abdicates', ['AE', 'B', 'D', 'AH', 'K', 'EY', 'T', 'S']],
 ['abdicating', ['AE', 'B', 'D', 'IH', 'K', 'EY', 'T', 'IH', 'NG']],
 [

In [20]:
x = np.array([_x[0] for _x in data])

In [21]:
y = np.array([_y[1] for _y in data])

In [22]:
xlen = np.array([len(_x) for _x in x])

*The largest word is*

In [23]:
np.argmax(xlen)

22468

In [24]:
x[22470]

'unchartered'

In [30]:
X = []

for eachword in x:
    reword = [ord(c)-ord('a') for c in list(eachword)]
    
    vword = reword + [ord('z')-ord('a')+1]*(24-len(reword))
    vmask = [1]*len(reword) + [0]*(24-len(reword))
    X.append(np.array([vword, vmask]).T)
    
X = np.array(X)

array([[list([0, 1, 0, 2, 10, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26]),
        list([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
       [list([0, 1, 0, 13, 3, 14, 13, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26]),
        list([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
       [list([0, 1, 0, 13, 3, 14, 13, 4, 3, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26]),
        list([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
       ...,
       [list([25, 14, 14, 12, 4, 3, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26]),
        list([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
       [list([25, 14, 14, 12, 8, 13, 6, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26]),
        list([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [29]:
X.reshape((24167, 26, 1))

ValueError: cannot reshape array of size 48334 into shape (24167,26,1)

In [15]:
X.shape

(24167, 24, 2)

The idea of this approach is that, a word is a fixed size vector of 24 rows and 2 columns. The first column is the word represented in character ordinates. The padding for the fixed size is filled with a new character with ord = 26. This represents no-characters/word termination.

The other column is the mask that tells which rows of the word vector are valid characters of the word and which are padding. 1 - character from word / 0 - padding.

The following approach is to run CNN and generate an autoencoder that can regenerate words with highest accuracy with minimal representation. The logic is that, this current representation is extremely sparse. But if we can auto encode it into a single vector that upon decoding regenerates the word as it is.

Once this is fully functional, we can plug the encoder part into another CNN to learn the number of phonemes based on the word. This will be the first step towards phonetics prediction.  Using the encoded representation and predicted phonetic representation size, we can build an LSTM that can generate the actual phonetic representation.

+ Major flaw in this approach is that - the idea of using numeric representation for characters imply scale invariance. That is, 'aback' is as legitimate as 'bcbdl' based on cosine similarity.

+ The complete on-hot representation would be way too sparse for a CNN to work with and generate some reasonable internal representation.

> This trial is expected to fail. But an attempt is always worth it.

In [16]:
from keras.callbacks import TensorBoard
import tensorflow as tf

from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K

with tf.device('GPU:0'):
    input_img = Input(shape=(24, 2, 1))  # adapt this if using `channels_first` image data format

    x = Conv2D(64, (, 1), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((1, 1), padding='same')(x)
    x = Conv2D(64, (1, 1), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((1, 1), padding='same')(x)
    x = Conv2D(64, (1, 1), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((1, 1), padding='same')(x)
    x = Conv2D(64, (1, 1), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((1, 1), padding='same')(x)
    
    
    encoded = MaxPooling2D((1, 1), padding='same')(x)

    # at this point the representation is (4, 4, 8) i.e. 128-dimensional
with tf.device('GPU:1'):
    x = Dense()
    x = Conv2D(16, (3, 2), activation='relu', padding='same')(encoded)
    x = UpSampling2D((1, 1))(x)
    x = Conv2D(32, (2, 2), activation='relu', padding='same')(x)
    x = UpSampling2D((1, 1))(x)
    x = Conv2D(64, (1, 2), activation='relu', padding='same')(x)
    x = UpSampling2D((1, 1))(x)
    decoded = Conv2D(1, (3, 2), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_img, decoded)
    autoencoder.summary()
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')

TypeError: __init__() missing 1 required positional argument: 'units'

In [104]:
with tf.device('GPU:1'):
    autoencoder.fit(X, X,
                    epochs=100,
                    batch_size=16,
                    shuffle=True,
                    validation_data=(X, X),
                    callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])

Train on 24169 samples, validate on 24169 samples
Epoch 1/100
24169/24169 [==============================] - 19s 778us/step - loss: 233.8290 - val_loss: 233.7365
Epoch 2/100
24169/24169 [==============================] - 18s 746us/step - loss: 233.7365 - val_loss: 233.7365
Epoch 3/100
24169/24169 [==============================] - 18s 762us/step - loss: 233.7365 - val_loss: 233.7365
Epoch 4/100
24169/24169 [==============================] - 18s 756us/step - loss: 233.7365 - val_loss: 233.7365
Epoch 5/100
24169/24169 [==============================] - 19s 789us/step - loss: 233.7365 - val_loss: 233.7365
Epoch 6/100
24169/24169 [==============================] - 18s 758us/step - loss: 233.7365 - val_loss: 233.7365
Epoch 7/100
24169/24169 [==============================] - 18s 738us/step - loss: 233.7365 - val_loss: 233.7365
Epoch 8/100
24169/24169 [==============================] - 19s 797us/step - loss: 233.7365 - val_loss: 233.7365
Epoch 9/100
24169/24169 [=============================

KeyboardInterrupt: 